# <span style ='color:purple'>Import & settings

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%reload_ext autoreload
%autoreload 2

from data_provider.data_factory import data_provider
from tabular.dl_logs import logs_summary, exp_summary
from tabular.data import load_data
from tabular.tabular_models import compare_models_w_hpo

In [2]:
from types import SimpleNamespace
lt_logs_df = pd.read_csv(os.path.join('./logs/long_term_forecast.csv'), index_col = 0)
usc_args = SimpleNamespace(**lt_logs_df.loc[207].to_dict())

In [3]:
seq_len = 32
pred_len = seq_len
usc_args.seq_len = seq_len
usc_args.pred_len = pred_len
usc_args.scale = False

# <spn style = 'color:lightgreen'>DL Logs 

In [4]:
ldf = logs_summary(seq_len = seq_len)
ldf

test 4233


,model,features,fde,rmse
0,ST_GAT,MS,2.824565,1.696490
1,SABFormer,MS,2.849299,1.627774
2,Transformer,MS,2.931722,1.691967
3,iTransformer,S,2.937358,1.654779
4,Transformer,S,2.939580,1.653016
5,TimesNet,S,2.956967,1.665073
6,PatchTST,S,2.964681,1.676879
7,Nonstationary_Transformer,S,2.968956,1.694405
8,LSTM,MS,2.999465,1.800995
9,DLinear,S,3.018873,1.695828


# <span style ='color:cyan'> Data loading

In [5]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data(seq_len=seq_len, pred_len=pred_len, off_mask=False, downsample_factor=8)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((39389, 24), (39389,), (8440, 24), (8440,), (8441, 24), (8441,))

# <span style = 'color:pink'>Model fitting

In [9]:
comp_df = compare_models_w_hpo(X_train=X_train, y_train=y_train,X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test,
                               cv = False, n_iter_ml=20, n_iter_ann=1, method='regression')

Tested Linear Regression - reached RMSE of 3.018 over the test set
Tested ElasticNet - reached RMSE of 3.589 over the test set
Tested SGD  - reached RMSE of 4.222 over the test set
Tested kNN - reached RMSE of 3.809 over the test set
Tested Support Vector - RBF - reached RMSE of 3.574 over the test set
Tested Decision Tree - reached RMSE of 3.714 over the test set
Tested Random Forest - reached RMSE of 3.523 over the test set
Tested ADA Boost  - reached RMSE of 3.713 over the test set


/home/admin/.pyenv/versions/3.10.6/envs/tsl_env/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/admin/.pyenv/versions/3.10.6/envs/tsl_env/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/admin/.pyenv/versions/3.10.6/envs/tsl_env/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/admin/.pyenv/versions/3.10.6/envs/tsl_env/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/admin/.pyenv/versions/3.10.6/envs/tsl_env/lib/python3.10/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered

Tested Gradient Boosting - reached RMSE of 3.552 over the test set
Tested XGB  - reached RMSE of 3.531 over the test set


In [10]:
comp_df

,Best hyperparameters,RMSE
Model,,
Linear Regression,{},3.018366
Random Forest,"{'criterion': 'squared_error', 'max_depth': 15...",3.523409
XGB,"{'eta': 0.13462707592097856, 'gamma': 0.075639...",3.531214
Gradient Boosting,"{'learning_rate': 0.07540248655398828, 'max_de...",3.551671
Support Vector - RBF,"{'C': 56.119244014400074, 'gamma': 0.002682180...",3.574463
ElasticNet,"{'alpha': 0.0015442257663942972, 'l1_ratio': 0...",3.589290
ADA Boost,"{'learning_rate': 0.0026872109384420786, 'n_es...",3.713393
Decision Tree,"{'max_depth': 3, 'min_samples_leaf': 30, 'min_...",3.713767
kNN,{'n_neighbors': 56},3.808607
